# Dati Coronavirus Internazionali

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from dateutil import relativedelta
from IPython.display import Markdown
import plotly.express as px
import plotly.io as pio
import dateutil.relativedelta

pio.renderers.default = 'notebook_connected'
pio.templates.default = "simple_white+gridon"
plt_config = {'scrollZoom':False}

In [ ]:
who_url = "https://covid19.who.int/WHO-COVID-19-global-data.csv"
who = pd.read_csv(who_url, parse_dates=['Date_reported'], 
                   index_col='Date_reported')


In [ ]:
who = who.rename(columns={' Country_code':'country_code', 
                          ' Country':'country',
                          ' WHO_region':'region',
                          ' New_cases':'new_cases',
                          ' Cumulative_cases':'cases',
                          ' New_deaths':'new_deaths',
                          ' Cumulative_deaths':'deaths'})
updated_at = who.index.max()

In [ ]:
display(Markdown(f"### *Aggiornamento al {updated_at:%d/%m/%Y}*"))

cases = who.new_cases.resample('D').sum()
deaths = who.new_deaths.resample('D').sum()
display(Markdown(f"""

*Casi totali*: **{cases.sum():,d}**, *Decessi totali*: **{deaths.sum():,d}**

"""))
totals = pd.DataFrame(data={'cases': cases, 'deaths': deaths})
fig = px.bar(totals, title="andamento totale casi e decessi")
fig.update_xaxes(rangeslider_visible=True, title='Data')
fig.update_yaxes(title='Numero persone')
fig.show(config = plt_config)

In [ ]:
cases_by_country = who.groupby("country").last().nlargest(20, columns=['cases']).cases
deaths_by_country = who.groupby("country").last().nlargest(20, columns=['deaths']).deaths
fig = px.bar(cases_by_country, title='casi per nazione')
fig.update_layout(showlegend=False, yaxis_fixedrange = True )
fig.update_xaxes(title='Nazione')
fig.update_yaxes(title='Numero casi')

fig.show(config=plt_config)

In [ ]:
fig = px.bar(deaths_by_country, title='decessi per nazione')
fig.update_layout(showlegend=False, yaxis_fixedrange = True )
fig.update_xaxes(title='Nazione')
fig.update_yaxes(title='Numero decessi')
fig.show(config = plt_config)

In [ ]:
today = datetime.now()
last_month = today - dateutil.relativedelta.relativedelta(months=1)
last = who[who.index > last_month].query('new_cases>1000').sort_values('new_cases', ascending=False)
fig = px.bar(last, x=last.index, y=last.new_cases, color='country', 
       title='casi > 1.000 per giorno (ultimo mese)')
fig.update_xaxes(title='Data')
fig.update_yaxes(title='Numero casi')

fig.show(config = plt_config)

In [ ]:
last = who[who.index > last_month].query('new_deaths>300').sort_values('new_deaths', ascending=False)
fig = px.bar(last, x=last.index, y=last.new_deaths, color='country', 
       title='decessi > 300 per giorno (ultimo mese)')
fig.update_xaxes(title='Data')
fig.update_yaxes(title='Numero decessi')

fig.show(config = plt_config)

*fonte*: [Dati WHO](https://covid19.who.int/WHO-COVID-19-global-data.csv)